In [1]:
# hello -> 你好
# seq -> seq
# Transformer
from transformers import AutoTokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-ro', use_fast=True)
print(tokenizer)

D:\.venv\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SupercoldZzz\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-ro', vocab_size=59543, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'})


D:\.venv\lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [3]:
tokenizer.batch_encode_plus([['hello, everyone today is a good day', 'It is late, please go home']])

{'input_ids': [[92, 778, 3, 1773, 879, 32, 8, 265, 431, 84, 32, 1450, 3, 709, 100, 540, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [4]:
from datasets import load_dataset

dataset = load_dataset(path='wmt16', name='ro-en')

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/610320 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1999 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1999 [00:00<?, ? examples/s]

Dataset wmt16 downloaded and prepared to C:/Users/SupercoldZzz/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 610320
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1999
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 1999
    })
})

In [6]:
# 采样
dataset['train'] = dataset['train'].shuffle(1).select(range(20000))
dataset['validation'] = dataset['validation'].shuffle(1).select(range(200))
dataset['test'] = dataset['test'].shuffle(1).select(range(200))

In [7]:
dataset['train'][0]

{'translation': {'en': 'For these reasons I voted in favour of the proposal for a new regulation that aims for greater clarity and transparency in the GSP system.',
  'ro': 'Din aceste motive am votat în favoarea propunerii de nou regulament care își propune o mai mare claritate și transparență în sistemul SPG.'}}

In [8]:
def preprocess_function(data, tokenizer):
    en = [ex['en'] for ex in data['translation']]
    ro = [ex['ro'] for ex in data['translation']]
    
    data = tokenizer.batch_encode_plus(en, max_length=128, truncation=True)
    
    with tokenizer.as_target_tokenizer():
        data['labels'] = tokenizer.batch_encode_plus(
        ro, max_length=128, truncation=True)['input_ids']
        
    return data

In [10]:
dataset = dataset.map(preprocess_function,
           batched=True,
           batch_size=1000,
           num_proc=1,
           remove_columns=['translation'],
           fn_kwargs={'tokenizer': tokenizer})

  0%|          | 0/20 [00:00<?, ?ba/s]

D:\.venv\lib\site-packages\transformers\tokenization_utils_base.py:3582: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
print(dataset['train'][0])

{'input_ids': [460, 354, 3794, 12, 10677, 20, 5046, 14, 4, 2546, 37, 8, 397, 5551, 30, 10113, 37, 3501, 19814, 18, 8465, 20, 4, 44690, 782, 2, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [902, 576, 2946, 76, 10815, 17, 5098, 14997, 5, 559, 1140, 43, 2434, 6624, 27, 50, 337, 19216, 46, 22174, 17, 2317, 121, 16825, 2, 0]}


In [14]:
def collate_fn(data):
    # 求最长的label
    max_length=max([len(i['labels']) for i in data])
    
    for i in data:
        pads = [-100] * (max_length - len(i['labels']))
        i['labels'] = i['labels'] + pads
        
    data = tokenizer.pad(
        encoded_inputs=data,
        padding=True,
        max_length=None,
        pad_to_multiple_of=None,
        return_tensors='pt'
    )
    
    # decoder_input_ids
    data['decoder_input_ids'] = torch.full_like(data['labels'], 
                                               tokenizer.get_vocab()['pad'],
                                               dtype=torch.long)
    data['decoder_input_ids'][:, 1:] = data['labels'][:, :-1]
    data['decoder_input_ids'][data['decoder_input_ids'] == -100] = tokenizer.get_vocab()['<pad>']
    return data

In [16]:
import torch

loader = torch.utils.data.DataLoader(
    dataset=dataset['train'],
    batch_size=8,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True
)

for data in loader:
    break

In [17]:
data

{'input_ids': tensor([[   12,  1107,    30,    37,     4,  2194,   476,    63,   123,    47,
           116,    15, 27384,  1036,     3,    18,    66,     8,  9911,  1591,
           141,     2,     0, 59542, 59542, 59542, 59542, 59542, 59542, 59542,
         59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542,
         59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542,
         59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542,
         59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542,
         59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542,
         59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542],
        [   84,    32,   154,  2129,     8,  1348,    14,  2868,  6607,  1618,
            13,   696,  5746,     2, 19106,     3,    18,   561,    12,    76,
          4754,   108,     4,    15, 38405,  4997,    14,     4,   346,     7,
            11,  1831,     3,   350,  5985, 

In [18]:
for k, v in data.items():
    print(k, v.shape)

input_ids torch.Size([8, 89])
attention_mask torch.Size([8, 89])
labels torch.Size([8, 103])
decoder_input_ids torch.Size([8, 103])


In [19]:
from transformers import AutoModelForSeq2SeqLM, MarianModel

In [35]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained = MarianModel.from_pretrained('Helsinki-NLP/opus-mt-en-ro')
        
        self.register_buffer('final_logits_bias', torch.zeros(1, tokenizer.vocab_size))
    
        self.fc = torch.nn.Linear(512, tokenizer.vocab_size, bias=False)
        
        # 加载预训练权重参数
        parameters = AutoModelForSeq2SeqLM.from_pretrained('Helsinki-NLP/opus-mt-en-ro')
        self.fc.load_state_dict(parameters.lm_head.state_dict())
        self.criterion = torch.nn.CrossEntropyLoss()
        
    def forward(self, input_ids, attention_mask, labels, decoder_input_ids):
        logits = self.pretrained(input_ids=input_ids,
                                attention_mask=attention_mask,
                                decoder_input_ids=decoder_input_ids)
        logits = logits.last_hidden_state
        logits = self.fc(logits) + self.final_logits_bias
        loss = self.criterion(logits.flatten(end_dim=1), labels.flatten())
        return {'loss': loss, 'logits': logits}

In [36]:
model = Model()

Some weights of the model checkpoint at Helsinki-NLP/opus-mt-en-ro were not used when initializing MarianModel: ['final_logits_bias']
- This IS expected if you are initializing MarianModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MarianModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
print(sum(i.numel() for i in model.parameters()))

105634816


In [28]:
out = model(**data)
out['loss'], out['logits'].shape

(1.4804006814956665, torch.Size([8, 103, 59543]))

In [43]:
def test(model):
    loader_test = torch.utils.data.DataLoader(
        dataset=dataset['test'],
        batch_size=8,
        collate_fn=collate_fn,
        shuffle=True,
        drop_last=True
    )
    predictions = []
    references = []
    for i, data in enumerate(loader_test):
        with torch.no_grad():
            out = model(**data)
            
        pred = tokenizer.batch_decode(out['logits'].argmax(dim=2))
        label = tokenizer.batch_decode(data['decoder_input_ids'])
        predictions.append(pred)
        references.append(label)
        
        if i % 2 == 0:
            print(i)
            input_ids = tokenizer.decode(data['input_ids'][0])
            print('input_ids=', input_ids)
            print('pred=', pred[0])
            print('label=', label[0])
            
        if i == 10:
            break
            
    references = [[j] for j in references]


In [30]:
test(model)

0
input_ids= The▁only name that▁was not▁mentioned by▁any of the▁participants in the▁negotiations of recent▁days is that of the▁former▁head of the▁branch,▁Mayor Gheorghe Nichita.</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
pred= Singurul nume care nu a fost men de niciunul dintre participanții la negocierile din ultimele zile este cel al fostului șef al filialei, primarul Gheorghe Nichita.</s> DEaa al al Nicol Nicol Nicol N N În În În În În În În În În În În În În În În În În În Singur Singur Singur Singur Singur Singur
label= pad Singurul nume care nu a fost menționat de niciunul din participanții la negocierile din ultimele zile este cel al fostului lider al filialei, primarul Gheorghe Nichita.</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <

In [31]:
from transformers import AdamW
from transformers.optimization import get_scheduler

In [32]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [39]:
def train():
    optimizer = AdamW(model.parameters(), lr=2e-5)
    scheduler = get_scheduler(name='linear',
                             num_warmup_steps=0,
                             num_training_steps=len(loader),
                             optimizer=optimizer)
    model.to(device)
    model.train()
    
    for i, data in enumerate(loader):
        for k in data.keys():
            data[k] = data[k].to(device)
        out = model(**data)
        loss = out['loss']
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        optimizer.zero_grad()
        model.zero_grad()
        
        if i % 50 == 0:
            out = out['logits'].argmax(dim=2)
            correct = (data['decoder_input_ids'] == out).sum().item()
            total = data['decoder_input_ids'].shape[1] * 8
            accuracy = correct / total
            
            predictions = []
            references = []
            
            for j in range(8):
                pred = tokenizer.decode(out[j])
                label = tokenizer.decode(data['decoder_input_ids'][j])
                predictions.append(pred)
                references.append(label)
                
            lr = optimizer.state_dict()['param_groups'][0]['lr']
            print(i, loss.item(), accuracy, lr)

In [40]:
train()

0 2.2159132957458496 0.0 1.9992e-05
50 0.5485211610794067 0.0 1.9592e-05
100 1.1590079069137573 0.006147540983606557 1.9192000000000002e-05
150 0.9607174396514893 0.002777777777777778 1.8792000000000002e-05
200 0.9061514735221863 0.0011261261261261261 1.8392e-05
250 0.8414883613586426 0.0 1.7992e-05
300 0.5579505562782288 0.0 1.7592000000000004e-05
350 0.8098045587539673 0.01524390243902439 1.7192e-05
400 0.7466010451316833 0.009259259259259259 1.6792e-05
450 0.7285252809524536 0.006944444444444444 1.6392e-05
500 0.7612788081169128 0.005208333333333333 1.5992000000000002e-05
550 0.518362283706665 0.0023584905660377358 1.5592e-05
600 0.6960635781288147 0.010080645161290322 1.5192000000000003e-05
650 1.0835756063461304 0.005434782608695652 1.4792000000000002e-05
700 0.8726363182067871 0.004464285714285714 1.4392000000000002e-05
750 1.0110883712768555 0.004261363636363636 1.3992000000000001e-05
800 0.6736987829208374 0.0030864197530864196 1.3592000000000001e-05
850 1.0575217008590698 0.00

In [42]:
torch.save(model, '../data/翻译.model')

In [45]:
model2 = torch.load('../data/翻译.model', map_location='cpu')

In [46]:
test(model2)

0
input_ids= ▁Last▁month▁saw▁lowest▁growth▁rise▁since▁records▁began in 2000</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
pred= Luna ultima lună,-a înregistrat o mai mică creşte de 2000. în prezent.</s> țiululululul -: - De De De De De De De De De De De De De De De De Luna De De De De De De Luna De Luna Luna Luna Luna De Luna Luna De Luna Luna Luna Luna De Luna De Luna Luna Luna Luna Luna Luna Luna Luna
label= pad În ultima lună s-a înregistrat cea mai lentă creștere din 2000 până în prezent.</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 